In [1]:
#Run Model
import pickle

filename = 'data_90023.sav'

with open(filename, 'rb') as file:
    loaded_model = pickle.load(file)


In [ ]:
#Coefficients should be draw values, setpiont, upper energy, l,ower energy, hp energy, upper nodes,lower nodes
import numpy as np
M = loaded_model.coef_
window = 8

N = [[ele[window:2*window]] for ele in M] #Setpoints
N = np.array(N)
M_new = [np.hstack([*x[0:window], *x[2*window:]]) for x in M]

M_new




[array([-0.00326269,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.47095041,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.33127396,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.68961387,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.92045007, -0.06975608]),
 array([-0.00695913, -0.00680217,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  1.64432493, -0.47162231,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.60738041, -0.09767575,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.9853621 ,
        -0.12489564,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.84265364, -0.00417374]),


In [8]:
#8 hour horizon model Data Driven Model
import cvxpy as cp
import numpy as np

i = 8 #horizon
lambda_ = 1
l = 0.75 #soft constraint
n = 8
T_min = 40
T_max = 60

#example value
#model

def getOptimalSetpoint(x): #input values [d1...d8, twh_3,]

  # Construct the problem.
  #declare variables
  T = cp.Variable(i) #Temperatures 
  s = cp.Variable(i) #setpoint variables
  z = cp.Variable(i) #unmet demand
  objective = cp.Minimize(cp.sum(lambda_ * z**2 + (lambda_*l) * s))

  constraints = [
      s >= 49,
      s <= T_max,
      T == M_new @ x + N @ s,
      z >= T_min - T,
      z >= 0
      ]
  prob = cp.Problem(objective, constraints)

  # The optimal objective value is returned by `prob.solve()`.
  result = prob.solve()

  # Print results
  print("Optimal T values:", T.value)
  print("Optimal s values:", s.value)
  print("Optimal objective value:", result)

  return s

In [9]:
x = [0, 50, 0, 50, 0, 50, 0,  0, 39.41643795734582,39.41643795734583]
getOptimalSetpoint(x)

Optimal T values: [38.18518536 35.9555718  35.80395794 32.88139063 32.62941992 29.50352889
 30.66342014 31.15384023]
Optimal s values: [60. 49. 60. 60. 60. 60. 49. 60.]
Optimal objective value: 761.3599427138514


Variable((8,), var362)

In [15]:
#import data
import pandas as pd
import numpy as np

#Data transformation, Y-values are y_i+1 temperatures for the next hour
def rolling_subsets(input_list, window_size = 4):
    if window_size > len(input_list) or window_size <= 0:
        raise ValueError("Window size must be positive and not greater than the list length.")
    
    return [input_list[i:i + window_size] for i in range(int(len(input_list)))]

import numpy as np

def csv_to_features(df, type = 1): #exports features, y = 15 minute increment predictions for 1 hour of outlet temperature
        
    y_col = df["Hot Water Outlet Temperature (C)"]
    window = 8 #2 hours
    result = rolling_subsets(y_col, window)

    power = df["Average Electric Power"]
   

    if type == 2: #EWH
        # Create feature vectors
        T_u = [value if mode == "Upper On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
        T_m = [value if mode == "Lower On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
        xu = rolling_subsets(T_u, window)
        xl = rolling_subsets(T_m, window)

        x2 = rolling_subsets(df["Draw Data (L)"], window)
        x3 = rolling_subsets(df["Setpoint (C)"], window)
        x4 = df["T_WH3"]
        x5 = df["T_WH10"]

    else: #HPWH
        # Create feature vectors
        T_u = [value if mode == "Upper On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
        T_m = [value if mode == "Lower On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
        T_h = [value if mode == "Heat Pump On" else 0 for value, mode in zip(df["Average Electric Power"], df["Water Heating Mode"])]
        
        xu = rolling_subsets(T_u, window)
        xl = rolling_subsets(T_m, window)
        xh = rolling_subsets(T_h, window)
        
        x2 = rolling_subsets(df["Draw Data"], window)
        x3 = rolling_subsets(df["Setpoint"], window)
        x4 = df["T_WH3"]
        x5 = df["T_WH10"]
        xh_v = xh[:-window]

    print(len(result), len(x2), len(x3))


    y = result[:-window]  # Remove the first index to offset, y = i+1th prediction
    xl_v = xl[:-window]  # Water Heater Power
    xu_v = xu[:-window]

    x2_v = x2[:-window] #draw values
    x3_v = x3[:-window] #setpoint values
    x4_v = x4[:-window] #noe temperatures should only be taken at current time interval (live readings)
    x5_v = x5[:-window]


    # Stack features
    if type == 2:
        X = np.column_stack([x2_v, xu_v, xl_v, x4_v, x5_v]) 
    else:
        X = np.column_stack([x2_v, xu_v, xl_v, xh_v, x4_v, x5_v]) #Draw value, upper node powr, lower node power, heat pump power, upper node temp, lowe rnode temp
    return X, y


In [16]:
#Validate
type = 1 

if type == 1: # HPWH
    data = pd.read_csv("output_site_90023.csv")

else: #EWH

    data = pd.read_csv("output_site_90023_ewh.csv")
    
    
X, y = csv_to_features(data, type)

929236 929236 929236


In [21]:
prob.solve()

NameError: name 'prob' is not defined

In [19]:
#validation test


X_val_filtered = []
y_val_filtered = []
for x, y in zip(X, y):
    if all(val >= 40 for val in y):
        X_val_filtered.append(x)
        y_val_filtered.append(y)
X_val = X_val_filtered
y_val = y_val_filtered

from sklearn.linear_model import LinearRegression

#error metrics
from sklearn.metrics import r2_score



In [36]:
# Construct the problem.
#declare variables
x = np.array(X[0]).flatten()

T = cp.Variable(i) #Temperatures 
s = cp.Variable(i) #setpoint variables
z = cp.Variable(i) #unmet demand
objective = cp.Minimize(cp.sum(lambda_ * z**2 + (lambda_*l) * s))

constraints = [
    s >= 49,
    s <= T_max,
    T == M_new @ x + N @ s,
    z >= T_min - T,
    z >= 0
    ]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.



In [32]:
M_new @ x + N @ s

Expression(AFFINE, UNKNOWN, (8, 8))

In [34]:
import inspect
print(inspect.signature(prob.solve))

(*args, **kwargs)


In [38]:
result = prob.solve(solver = cp.ECOS)

# Print results
print("Optimal T values:", T.value)
print("Optimal s values:", s.value)
print("Optimal objective value:", result)

SolverError: The solver ECOS is not installed.

In [ ]:
#Query Ochre
import os
import datetime as dt
import pandas as pd
import numpy as np

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files
from ochre import HeatPumpWaterHeater



def simulate_ochre():
    # Define equipment and simulation parameters
    setpoint_default = 51  # in C
    deadband_default = 5.56  # in C
    max_setpoint = 60
    min_setpoint = 40

    run_range = False #runs simulation for a variety of setpoints specified in setpoint_range
    simulation_days = 1 #172 #220
    site_number = 90023#10292#'10441'

    site_number = 'null'

    flow_data = f'net_flow_{site_number}.csv'

    #start_date = dt.datetime(2013, 1, 17, 0, 1) #10441
    start_date = dt.datetime(2013, 1, 1, 0, 1) #10292, 90023

    setpoint_range = [setpoint_default]

    if run_range == True:
        setpoint_range = np.arange(min_setpoint, max_setpoint, 0.5)

    x = [0, 0, 0, 0, 0, 0, 0, 0, 51, 51]
    setpoint_pred = getOptimalSetpoint(x)

    for s in setpoint_range: #run simulation for every setpoint in valid range
        setpoint_default = s
        print("Simulating Setpoint: ", setpoint_pred)
        equipment_args = {
            "start_time": start_date,  # year, month, day, hour, minute
            "time_res": dt.timedelta(minutes=1),
            "duration": dt.timedelta(minutes=120),
            "verbosity": 9,  # required to get setpoint and deadband in results
            "save_results": False,  # if True, must specify output_path
            # "output_path": os.getcwd(),        # Equipment parameters
            "Setpoint Temperature (C)": setpoint_default,
            "Tank Volume (L)": 250,
            "Tank Height (m)": 1.22,
            "UA (W/K)": 2.17,
            "HPWH COP (-)": 4.5,
        }

        # Create water draw schedule
        times = pd.date_range(
            equipment_args["start_time"],
            equipment_args["start_time"] + equipment_args["duration"],
            freq=equipment_args["time_res"],
            inclusive="left",
        )
        water_draw_magnitude = 12  # L/min
        #withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
        withdraw_rate = np.loadtxt(f'ochre\defaults\\Input Files\\{flow_data}')
        withdraw_rate = withdraw_rate[:len(times)]
        schedule = pd.DataFrame(
            {
                "Water Heating (L/min)": withdraw_rate,
                "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
                "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
                "Zone Temperature (C)": 20,
                "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
                "Mains Temperature (C)": 7,
            },
            index=times,
        )

        # Initialize equipment
        hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

        # Simulate
        data = pd.DataFrame()
        data = {'draw_data' :[], 'setpoint' :[]}
        control_signal = {}
        setpoints = []

        #generate noise for setpoint profile
        noise = np.random.randint(0, 3, 24, dtype=np.int8)
        for t in hpwh.sim_times:
            # Change setpoint based on hour of day
            setpoint = setpoint_default

            setpoint_pred = getOptimalSetpoint(x)
            control_signal = {
                "Setpoint": setpoint_pred[0]
            }
            setpoints.append(setpoint)
            # Run with controls
            _ = hpwh.update(control_signal=control_signal)

        df = hpwh.finalize()

        cols_to_plot = [
            "Hot Water Outlet Temperature (C)",
            "Hot Water Average Temperature (C)",
            "Water Heating Deadband Upper Limit (C)",
            "Water Heating Deadband Lower Limit (C)",
            "Water Heating Electric Power (kW)",
            "Hot Water Unmet Demand (kW)",
            "Hot Water Delivered (L/min)",
        ]

        cols_to_save = [
            "Hot Water Outlet Temperature (C)",
            "T_WH3",
            "T_WH10"
        ]


        withdraw_rate = np.convolve(withdraw_rate, np.ones(15)/15, 'same')

        # Calculate the rolling average for 'setpoints' with window size 15
        setpoints = np.convolve(setpoints, np.ones(15)/15, 'same')

        # For the DataFrame, select columns and calculate the rolling average for each column
        to_save = df[cols_to_save].rolling(window=15).mean()

        draw_data = withdraw_rate[14::15]
        setpoints = setpoints[14::15]
        to_save = df.loc[:, cols_to_save]
        to_save = to_save[14::15]

        to_save["Draw Data"] = pd.Series(draw_data, index=to_save.index)
        to_save["Setpoint"] = pd.Series(setpoints, index=to_save.index)

        print(len(draw_data), len(setpoints), len(to_save))

        import matplotlib.pyplot as plt 

        df.loc[:, cols_to_plot].plot()
        print("Simulation Copmleted")
        return cols_to_save
        #to_save.to_csv(f'output_site_{site_number}.csv', mode='a', header=False, index=False)

    #plt.show()



In [3]:
#prbs
def prbs31(code, n):
    for i in range(n):
        next_bit = ~((code>>30) ^ (code>>27))&0x01
        code = ((code<<1) | next_bit) & 0xFFFFFFFF
    return code

In [ ]:
import pandas as pd
import numpy as np
T = pd.read_csv("output_site_90159.csv")
flow = T["Draw Data (L)"].unique()

T = pd.read_csv("output_site_90023.csv")
heavy_draw = T["Draw Data (L)"].unique()
heavy_draw = np.concatenate((heavy_draw, flow))
heavy_draw.sort()

In [ ]:

draws = np.random.choice(heavy_draw, 844784, p = prob)
draws = np.asarray(draws)
np.savetxt('large_draw.csv', draws)

In [ ]:
import random
#Generate heavy draw data, desired length is 844784
draw = []
draw_budget = 0
draw_prob = 100
for i in range(844784):
    random_int = random.randint(1, 100)
    if i % 1440 == 0:
        print(draw_budget)
        draw_budget = 0
    if random_int >= draw_prob:
        draw_value = np.random.choice(heavy_draw, 1)[0]
        draw.append(draw_value)
        draw_budget += draw_value
        draw_prob = 100
    else:
        draw.append(0)
    draw_prob -= 0.1

np.savetxt('net_flow_large.csv', draw)

0
22.926926188960003
2.4694525912066667
12.040748213906667
20.494293019020002
4.181823182413334
10.309967912066668
0.757082
13.62640094356
1.9647312578733334
36.51758276769335
4.938905182413333
5.587679031493334
9.535551257873333
24.94581152229334
2.6676591824133333
4.181823182413334
7.642308729653334
1.46000992454
45.27764231493334
3.028328
6.561377333333334
5.299574
13.212653106953333
6.48881354724
3.280688666666667
4.2359772578733335
16.76303709448
7.769026591206667
13.807272880573336
14.888204276893333
15.555387836606666
11.608590666666665
13.644810654193332
13.284141836606668
17.41181094356
1.0094426666666667
6.200708515746666
9.607039987526665
0.5047213333333334
2.16293784908
29.722254478326672
3.1723805157466667
35.67060232740667
2.16293784908
5.533524956033334
12.671112352353333
2.92001984908
3.4788952578733334
1.9647312578733334
5.0829576981599995
2.271246
8.651751396320002
10.274223547240002
11.012895836606667
30.98405781166
11.536026880573335
8.507698880573335
14.63584361022